### 2.3.1 Enriching Word Vectors with Subword Information

#### 一. 介绍
1. 在此之前的embedding, 为单词表的每个单词赋予一个不同的向量. 单词向量之间没有参数共享, 也没有考虑过单词的形态. 这在形态意义丰富的语言中是个缺陷  
2. 此前的embedding只能给出训练集中出现过的单词的向量, 而这个文章可以根据单词的'subword'推测出训练集中不存在的单词的向量

#### 二. skip-gram负采样模型  
1. 给定一个长度为$W$的单词表, 目标是学习每个单词的向量表示.skip-gram采用最大化如下似然函数的方法$$\sum _{ t=1 }^{ T }{ \sum _{ c\in { C }_{ t } }^{  }{ \log { p\left( { w }_{ c }|{ w }_{ t } \right)  }  }  } $$其中, ${ w }_{ t }$为目标单词,${ w }_{ c }$为上下文  

2. 给定目标词,得到上下文词的概率可以假设为softmax概率, 即$$p\left( { w }_{ c }|{ w }_{ t } \right) =\frac { { e }^{ s\left( { w }_{ t },{ w }_{ c } \right)  } }{ \sum _{ j=1 }^{ W }{ { e }^{ s\left( { w }_{ t },{ w }_{ j } \right)  } }  } $$但是因为我们只想直到某一个上下文单词在给定目标单词后出现的概率是多少, 所以只要把问题相乘1个二分类问题. 从corpus中抽取目标词和上下文词作为正例, 目标词和随机选择单词表的词构成负例即可. 然后计算词向量.目标词和上下文的得分为 : $s\left( { w }_{ c }|{ w }_{ t } \right) ={ u }_{ { w }_{ c } }^{ T }*{ v }_{ { w }_{ t } }$

#### 三. subword模型
1. 使用子字表示单词  
 上述`skip-gram`没有考虑单词的形态, 这里我们将单词表示成一组'n个字符的集合'(bag of character n-gram).   
 为了区分单词的前后缀, 在单词开始和结尾分别加上符号'<'和'>'.因此, 单词'where'的3-gram表示为:   
 $<wh$ , $whe$ , $her$ , $ere$ , $re>$  
2. 单词的representation  
 假设我们得到了长度为$G$的`n-gram子字`(n个字符)词典.  
 假设一个单词w, 可以表示成子字集合${ g  }_{ w }=\left\{ 1,..,G \right\} $,集合中为单词的子字在字典中的index.  
  1. n的范围通常为[3,6]  
  2. 若每个子字g学习到一个向量$z_g$, 则单词$w$的向量表示为其子字集合${ g  }_{ w }$的和( sum ). 且两个单词w和c的得分函数 : $$s\left( w,c \right) =\sum _{ g\in { g  }_{ w } }^{  }{ { z }_{ g }^{ T }{ V }_{ c } } $$   
  3. 为了对子字单词表记忆, 使用hash函数, 将每一个子字转换成不同的integer数值.最终, 单词w被表示成其在word dictionary中的index和哈希后的子字数值集合
3. 使用子字模型训练词向量  
 同样采用上面的负采样二分类模型训练, 只是单词的表示变成了子字向量的和, 训练得到每个子字的表示

### 2.3.2 Bag of Tricks for Efficient Text Classification
本文讨论FastText用于标签预测和情感分类

#### 一. CBOW模型
1. CBOW和skip-gram是同时提出的向量化词的方法.都是基于2层神经网络的方法(1个隐藏层+1个分层softmax层)  
2. CBOW和skip-gram的区别在于  
  1. CBOW的输入层为target周围的n个context单词, 输出为target单词
  2. softmax输入为target单词, 输出为target单词周围n个位置, 有该context单词的概率
  
<img src='img/cbowandskipgram.png' height='60%' width='60%'>
3. CBOW和skip-gram是基于2层神经网络训练出来的, 而带负采样的skip-gram才是逻辑回归训练出来的
4. 比较各种方法训练出来的词向量好坏的一种方法是:   
 给定2组词, 看其相似度. 比如"man-women"相似于'king-queen'
 



#### 二. FastText模型
1. fasttext用于文本分类问题. fasttext得到的词向量是文本分类的副产物.
2. fasttext是有监督模型,和CBOW网络结构类似, 通过context预测标签. 与其区别主要在2个地方的优化:  
  1. fasttext直接进行文本分类, 因此fasttext的输出标签是文本类别, 而CBOW输出的是target word  
  2. fasttext使用分层softmax和n-gram优化训练效果  
3. 分层softmax:   
 1. 若使用传统softmax计算文本分类时, 当类别很多时计算量相当大. 时间复杂度为$O(kh)$. k为类别个数, h为文本向量长度  
 2. 若使用霍夫曼树构成的分层softmax, 则时间复杂度变为$O(hlog_2(k))$
 3. 分层softmax在测试阶段, 输出所属类别的概率时在速度上也很有优势.若节点深度为$l+1$,父节点为$\left( { n }_{ 1 }...{ n }_{ l } \right) $  
 则输出其类别概率为$$p(n_{l+1})=\prod _{ i=1 }^{ l }{ p\left( { n }_{ i } \right)  } $$  
4. 使用句子的n-gram表示局部信息  
 因为输入的训练数据是上下文的单词, 但这种输入彼此之间独立, 没有考虑词的顺序带来的语义问题.因此fastext在输入端使用n-gram模型把语句的局部顺序作为新特征一起训练. 具体的:  
   "I eat apple"的2元模型为: "I eat"和"eat apple"